In [1]:
import numpy as np

In [27]:
def match(a,b):
    sorted_indices = np.argsort(a)
    sorted_halo = a[sorted_indices]

    positions_in_sorted = np.searchsorted(sorted_halo, b)
    positions = sorted_indices[positions_in_sorted]
    
    return positions
def generate_points_on_unit_sphere(num_points):
    theta = np.random.uniform(0, 2 * np.pi, num_points)
    phi = np.random.uniform(0, np.pi, num_points)

    x = np.sin(phi) * np.cos(theta)
    y = np.sin(phi) * np.sin(theta)
    z = np.cos(phi)

    return x, y, z

## For Aemulus_nu

In [12]:
from halotools.sim_manager import CachedHaloCatalog
halocat = CachedHaloCatalog(fname='/home/wenhao/Halotools/HDF5/out_10/Aemulus_nu_0.hdf5',ptcl_version_name='rockstar_v1.53_no_cuts')

In [14]:
halocat.halo_table['halo_vrms']=(4.499*10**(-48))**0.5*(halocat.halo_table['halo_m200m']/halocat.halo_table['halo_r200m'])**0.5/(2**0.5)*3.09*10**19*(1+halocat.redshift)**0.5# Calculate 'halo_vrms' using Virial theorem.

In [16]:
from halotools.empirical_models import NFWPhaseSpace
nfw = NFWPhaseSpace(redshift=0.5375,prim_haloprop_key='halo_m200m',conc_mass_model='dutton_maccio14',mdef='200m')
halocat.halo_table['halo_nfw_conc']=nfw.conc_NFWmodel(table=halocat.halo_table)# Calculate concentration using dutton_maccio14 model providen by Halotools.

In [29]:
f_Gamma = 0.5+0.5*0.5
log_M_min = 12.5+1.5*0.5
sigma_log_M = 0.1+0.9*0.5
mcut = 10+3.5*0.5
msat = 13.5+1.5*0.5
alphasat = 0.2+1.8*0.5
eta_con = 0.2+1.8*0.5
etavc = 0.0+0.7*0.5
etavs = 0.2+1.8*0.5
fenv = -0.3+0.6*0.5
deltaenv = 0.5+1.5*0.5
sigmaenv = 0.1+0.9*0.5
gammaf = 0.5+1.0*0.5

In [24]:
from model import fiducial_model

from halotools.empirical_models import TrivialPhaseSpace
cens_occ_model = fiducial_model.Zcens(prim_haloprop_key='halo_m200m')
cens_occ_model.param_dict['f_Gamma']=f_Gamma
cens_occ_model.param_dict['logMmin']=log_M_min
cens_occ_model.param_dict['sigma_logM']=sigma_log_M
cens_occ_model.param_dict['fenv']=fenv
cens_occ_model.param_dict['deltaenv']=deltaenv
cens_occ_model.param_dict['sigmaenv']=sigmaenv
cens_prof_model = TrivialPhaseSpace(redshift=0.5375,mdef='200m',prim_haloprop_key='halo_m200m')

from halotools.empirical_models import BiasedNFWPhaseSpace
sats_occ_model = fiducial_model.ZSats(cenocc_model=cens_occ_model,prim_haloprop_key='halo_m200m')
sats_occ_model.param_dict['mcut']=mcut
sats_occ_model.param_dict['msat']=msat
sats_occ_model.param_dict['alphasat']=alphasat
sats_prof_model = BiasedNFWPhaseSpace(redshift=0.5375,prim_haloprop_key='halo_m200m',conc_mass_model='direct_from_halo_catalog',mdef='200m',halo_boundary_key='halo_r200m',concentration_key='halo_nfw_conc')
#sats_prof_model.param_dict['conc_gal_bias'] = eta_con 
halocat.halo_table['halo_nfw_conc']*=eta_con# If we use Biased NFW model providen by Halotools, there may be error when eta_con<1, hence I just multiply halo concentration with eta_con.

In [25]:
from halotools.empirical_models import HodModelFactory
model_instance = HodModelFactory(centrals_occupation = cens_occ_model, centrals_profile = cens_prof_model, satellites_occupation = sats_occ_model, satellites_profile = sats_prof_model)
model_instance.populate_mock(halocat,Num_ptcl_requirement=0)

In [30]:
model_instance.mock.galaxy_table['sigmav']=halocat.halo_table['halo_vrms'][match(halocat.halo_table['halo_id'],model_instance.mock.galaxy_table['halo_id'])]

model_instance.mock.galaxy_table['vx']=model_instance.mock.galaxy_table['halo_vx']*gammaf
model_instance.mock.galaxy_table['vy']=model_instance.mock.galaxy_table['halo_vy']*gammaf
model_instance.mock.galaxy_table['vz']=model_instance.mock.galaxy_table['halo_vz']*gammaf

galtype=model_instance.mock.galaxy_table['gal_type']
sigmav=model_instance.mock.galaxy_table['sigmav']

xyz_vc=generate_points_on_unit_sphere(len(sigmav[np.where(galtype=='centrals')]))

rand_vc=np.random.normal(np.zeros(len(sigmav[np.where(galtype=='centrals')])),etavc*sigmav[np.where(galtype=='centrals')])
model_instance.mock.galaxy_table['vx'][np.where(galtype=='centrals')]+=rand_vc*xyz_vc[0]
model_instance.mock.galaxy_table['vy'][np.where(galtype=='centrals')]+=rand_vc*xyz_vc[1]
model_instance.mock.galaxy_table['vz'][np.where(galtype=='centrals')]+=rand_vc*xyz_vc[2]

xyz_vs=generate_points_on_unit_sphere(len(sigmav[np.where(galtype=='satellites')]))

rand_vs=np.random.normal(np.zeros(len(sigmav[np.where(galtype=='satellites')])),etavs*sigmav[np.where(galtype=='satellites')])
model_instance.mock.galaxy_table['vx'][np.where(galtype=='satellites')]+=rand_vs*xyz_vs[0]
model_instance.mock.galaxy_table['vy'][np.where(galtype=='satellites')]+=rand_vs*xyz_vs[1]
model_instance.mock.galaxy_table['vz'][np.where(galtype=='satellites')]+=rand_vs*xyz_vs[2]

In [31]:
model_instance.mock.galaxy_table

halo_delta,halo_x,halo_vy,halo_m200m,halo_mvir,halo_y,halo_z,halo_r200m,halo_vx,halo_upid,conc_NFWmodel,halo_hostid,halo_id,halo_rvir,halo_vz,halo_num_centrals,halo_num_satellites,conc_gal_bias,gal_type,host_centric_distance,vy,x,conc_galaxy,vz,z,y,vx,sigmav
float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,int64,int64,float32,float32,int32,int32,float64,object,float64,float32,float32,float64,float32,float32,float32,float32,float64
5.349044,111.97436,-960.73346,42272000000000.0,42272000000000.0,20.32288,34.225445,0.79697305,-604.12396,-1,6.622914,52,52,0.79697305,319.47513,1,0,0.0,centrals,0.0,-1137.942,111.97436,0.0,82.43132,34.225445,20.32288,-508.68518,418.51750552821306
4.8268785,111.739845,-768.45337,46949000000000.0,46949000000000.0,16.812105,37.725998,0.82534003,-728.2059,-1,6.564723,64,64,0.82534003,-593.47766,1,0,0.0,centrals,0.0,-690.23376,111.739845,0.0,-597.0755,37.725998,16.812105,-432.0392,433.416807004198
4.726289,107.38376,-455.51416,68734000000000.0,68734000000000.0,15.564974,27.356504,0.93716496,-124.909836,-1,6.357612,80,80,0.93716496,1309.3868,1,0,0.0,centrals,0.0,-435.15225,107.38376,0.0,1276.6011,27.356504,15.564974,-106.56505,492.1378850773755
3.7550442,56.58715,-1053.3295,38487000000000.0,38487000000000.0,121.92078,62.359917,0.772435,390.8134,-1,6.675369,107,107,0.772435,307.2623,1,0,0.0,centrals,0.0,-1157.9402,56.58715,0.0,452.44595,62.359917,121.92078,515.71674,405.6346724615922
3.6620426,57.225025,-480.66104,17406000000000.0,17406000000000.0,120.95888,64.06986,0.592916,245.06339,-1,7.136042,117,117,0.592916,769.72015,1,0,0.0,centrals,0.0,-405.89465,57.225025,0.0,755.9589,64.06986,120.95888,133.23767,311.35935878302706
3.150774,57.700356,-195.66945,97535000000000.0,97535000000000.0,120.404854,66.58026,1.05312,-131.91306,-1,6.1732197,129,129,1.05312,-529.78235,1,0,0.0,centrals,0.0,244.97173,57.700356,0.0,-487.85364,66.58026,120.404854,-127.034515,553.0321130015317
3.5591192,33.773384,-152.22784,134310000000000.0,134310000000000.0,128.50078,7.371764,1.171652,488.1022,-1,6.009333,147,147,1.171652,421.77286,1,0,0.0,centrals,0.0,-167.7669,33.773384,0.0,413.22452,7.371764,128.50078,505.69397,615.2673847282695
3.3689322,32.997902,120.07776,36742000000000.0,36742000000000.0,79.47464,58.796967,0.760585,528.2428,-1,6.701469,168,168,0.760585,876.67474,1,0,0.0,centrals,0.0,103.2583,32.997902,0.0,907.6011,58.796967,79.47464,530.9476,399.4077851920165
3.679076,97.94979,-317.7347,204830000000000.0,204830000000000.0,7.797075,30.000263,1.348619,229.58537,-1,5.799785,213,213,1.348619,-23.798225,1,3,0.0,centrals,0.0,-188.76408,97.94979,0.0,82.3753,30.000263,7.797075,225.21796,708.2085676241305
